In [3]:
%pip install -r requirements.txt --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
# read data
business_data = pd.read_csv('business_coordinates.csv')
tree_df = pd.read_csv('Tree_Inventory_20240929.csv')

Clean up tree data for processing and rename some columns (longitude) and drop some unnecessary / repetitive ones:

In [4]:
# Process tree data csv
tree_data = tree_df

# get proper latitude and longitude data
from shapely.wkt import loads

# Convert WKT to shapely Point and extract latitude and longitude
tree_data['geometry'] = tree_data['GEOMETRY'].apply(loads)
tree_data['LATITUDE'] = tree_data['geometry'].apply(lambda point: point.y)
tree_data['LONGITUDE'] = tree_data['geometry'].apply(lambda point: point.x)

# drop unnecessary columns
tree_data = tree_data.drop(columns=['geometry', 'LONGTITUDE', 'New Georeferenced Column'])
tree_data['SPECIES'] = tree_data['SPECIES'].astype(str)

tree_data.head()

,GEOMETRY,SPECIES,DIAMETER,LATITUDE,LONGITUDE
0,POINT (-97.73398904092146 30.25239671647407),Live Oak,20.0,30.252397,-97.733989
1,POINT (-97.77949770772025 30.20704479399738),Ashe Juniper,8.0,30.207045,-97.779498
2,POINT (-97.73408544155564 30.252389392695715),Live Oak,31.0,30.252389,-97.734085
3,POINT (-97.73392463841283 30.252384583450418),Live Oak,21.0,30.252385,-97.733925
4,POINT (-97.73385944263053 30.252313660255723),Live Oak,20.0,30.252314,-97.733859


Getting the trees within a specified radius (set in kilometers) for each business - this one is slow

In [17]:
# Set radius for businesses to find local trees
from geopy.distance import geodesic
total_businesses = len(business_data)

radius = 1 # 1 km or other distance

# for each business, use the coordinates to get nearby trees within the specified radius
for idx, business_row in business_data.iterrows():

    # get location of business
    business_location = (business_row['lat'], business_row['long'])

    # initialize tree list for each business
    nearby_trees = []

    # for each tree, check if it is within the radius
    for _, tree_row in tree_data.iterrows():
        tree_location = (tree_row['LATITUDE'], tree_row['LONGITUDE'])
        distance = geodesic(business_location, tree_location).km

        # append within radius
        if distance <= radius:
            nearby_trees.append(tree_row['SPECIES'])

    # Store species of nearby trees
    business_data.at[idx, 'NearbyTreeSpecies'] = ', '.join(nearby_trees)

    # Print progress
    print(f"Processed business {idx + 1} of {total_businesses}")


Processed business 1 of 2808


KeyboardInterrupt: 

Faster way to get distances by converting stuff to radians?

In [5]:
from sklearn.neighbors import BallTree
import numpy as np

# Radius in kilometers
km = 1
radius = km / 6371.0  # Earth radius in km for haversine

# Prepare data for BallTree in radians
business_coords = np.radians(business_data[['lat', 'long']].values)
tree_coords = np.radians(tree_data[['LATITUDE', 'LONGITUDE']].values)

# Create BallTree for the tree locations
tree_ball_tree = BallTree(tree_coords, metric='haversine')

# Function to find nearby tree species for each business
def find_nearby_trees_for_business(business_location):
    # Query the tree BallTree for points within the radius
    indices = tree_ball_tree.query_radius([business_location], r=radius)[0]
    nearby_trees = tree_data.iloc[indices]['SPECIES'].tolist()
    return ', '.join(nearby_trees)

# Apply the function to each business
business_data['NearbyTreeSpecies'] = [find_nearby_trees_for_business(loc) for loc in business_coords]

# Optional: Print progress
print("Completed processing for all businesses.")


Completed processing for all businesses.


In [7]:
from sklearn.neighbors import BallTree
import numpy as np

# Radius in kilometers
radius = 1.0 / 6371.0  # Earth radius in km for haversine

# Prepare data for BallTree in radians
business_coords = np.radians(business_data[['lat', 'long']].values)
tree_coords = np.radians(tree_data[['LATITUDE', 'LONGITUDE']].values)

# Create BallTree for the tree locations
tree_ball_tree = BallTree(tree_coords, metric='haversine')

def find_nearby_trees_for_business(business_location):
    # Query the tree BallTree for points within the radius
    indices = tree_ball_tree.query_radius([business_location], r=radius)[0]

    # If no nearby trees are found within the radius, find the closest tree
    if len(indices) == 0:
        # Use the query method to find the nearest tree outside the radius
        nearest_index = tree_ball_tree.query([business_location], k=1)[1][0][0]
        nearest_tree = tree_data.iloc[nearest_index]['SPECIES']
        return f"{nearest_tree}"

    # Get species of nearby trees found within the radius
    nearby_trees = tree_data.iloc[indices]['SPECIES'].dropna().tolist()  # Remove NaNs if any
    return ', '.join([species for species in nearby_trees if species.strip()])  # Remove empty strings

# Apply the function to each business
business_data['NearbyTreeSpecies'] = [find_nearby_trees_for_business(loc) for loc in business_coords]
business_data.drop(columns=['Unnamed: 0'], inplace=True) # Drop unnecessary column
print(f'Finished processing {len(business_data)} businesses.')
business_data.head()

Finished processing 2808 businesses.


,Reported.Year,Property.ID..,Commercial.Property..Property.Street.Address,Building.Name,Total.SqFt,Category,Year.Built,kWh.sqft,Percentile.Rank,Received.Date,Site.EUI..kBTU.sqft.,Portfolio.Manager.SqFt,Total.Green.House.Gas.Emissions..MtCO2e.,Portfolio.Manager.Organization,GeoLocation,Submitted.Name.of.Organization,GeoLocation_clean,lat,long,NearbyTreeSpecies
0,2017.0,187746,"5126 E 5 ST, AUSTIN TX,","5126 E 5 ST, AUSTIN TX,",7419.0,NaN,1940,0.000,NaN,NaN,NaN,NaN,NaN,NaN,"5126 E 5 ST\nAUSTIN, TX\n(30.251707, -97.697313)","Pantur, Inc.","30.251707, -97.697313",30.251707,-97.697313,"Hackberry, Oak, Live (Southern), Hackberry, Oa..."
1,2017.0,230740,"2500 RIDGEPOINT DR, AUSTIN TX, 78728","2500 RIDGEPOINT DR, AUSTIN TX, 78728",49024.0,INDUSTRIAL (W/sqft > 3),1983,15.580,39.58,NaN,NaN,NaN,NaN,NaN,"2500 RIDGEPOINT DR\nAUSTIN, TX 78728\n(30.3280...",Equitable Commercial Realty,"30.328069, -97.672401",30.328069,-97.672401,"Oak, Live (Southern)"
2,2017.0,729528,"1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741","1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741",101953.0,STORE\GROCERY (W/sqft > 5),2009,32.063,76.26,NaN,NaN,NaN,NaN,NaN,"1700 S PLEASANT VALLEY RD\nAUSTIN, TX 78741\n(...","Mimco, Inc.","30.236494, -97.721013",30.236494,-97.721013,"Oak, Texas Live (Escarpment), Crapemyrtle, Cra..."
3,2017.0,526381,2011 W RUNDBERG LN Austin TX 78758,2011 W RUNDBERG LN Austin TX 78758,146142.0,OFFICE LG > 35000,1999,8.049,4.79,NaN,NaN,NaN,NaN,NaN,"2011 W RUNDBERG LN\nAustin, TX 78758\n(30.3737...",AISD,"30.373732, -97.719236",30.373732,-97.719236,"Oak, Texas Red, Oak, Post, Pecan, Sycamore, Me..."
4,2017.0,200001,"202 W 17 ST, AUSTIN TX, 78701","202 W 17 ST, AUSTIN TX, 78701",15350.0,OFFICE MED 10-35,1998,100.129,NaN,NaN,NaN,NaN,NaN,NaN,"202 W 17 ST\nAUSTIN, TX 78701\n(30.279196, -97...",Charter Communications,"30.279196, -97.740475",30.279196,-97.740475,"Chinese Tallow, Escarpment Live Oak, Pecan, Va..."


Save as csv if necessary

In [27]:
# save as csv
#business_data.to_csv('business_trees_1km.csv', index=False)

Decision Tree Era

In [77]:
from collections import Counter
import pandas as pd
import numpy as np

# Read data
data = pd.read_csv('business_trees_1km_with_weather_clusters.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)  # Drop redundant index col
data['NearbyTreeSpecies'] = data['NearbyTreeSpecies'].astype(str)

# Function to find the most common species
def get_most_common_species(nearby_species):
    if ', ' not in nearby_species:
        return nearby_species if nearby_species.strip() != 'Unknown' else np.nan 
    
    # Split the string into a list of species
    species_list = nearby_species.split(', ')
    species_counts = Counter(species_list)  # Count occurrences of each species
    
    most_common_species, _ = species_counts.most_common(1)[0]  # Get the most common species
    if most_common_species.strip() == 'Unknown':
        # Check if there is a second most common species
        if len(species_counts) > 1:
            # Get the second most common species
            second_most_common_species = species_counts.most_common(2)[1][0]
            return second_most_common_species
        else:
            # Return nan if all unknown
            return np.nan
    else:
        return most_common_species

# Apply the function to create a new column for the predicted species
data['BestTreeSpecies'] = data['NearbyTreeSpecies'].apply(get_most_common_species)

For any nan best tree species, fill with the mode of the respective cluster

In [78]:
unknown_mask = data[data['BestTreeSpecies'].isna()].index
print(f'There are still {unknown_mask.shape[0]} NA values for best tree species')

# Step 0: Ensure the column is cast to 'object' type to allow string entries
data['BestTreeSpecies'] = data['BestTreeSpecies'].astype('object')

# Step 1: Calculate the mode of BestTreeSpecies for each cluster
cluster_mode = data.groupby('cluster')['BestTreeSpecies'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

# Step 2: Fill missing values in BestTreeSpecies based on the mode for each cluster
data.loc[unknown_mask, 'BestTreeSpecies'] = data.loc[unknown_mask, 'cluster'].map(cluster_mode)

# Verify the result
unknown_now = data['BestTreeSpecies'].isna().sum()
print(f'There are now {unknown_now} NA values for BestTreeSpecies')

There are still 30 NA values for best tree species
There are now 0 NA values for BestTreeSpecies


encode for categories

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder


# Feature engineering
encoder = LabelEncoder()
data['Category_encoded'] = encoder.fit_transform(data['Category'])
data['BestTreeSpecies_encoded'] = encoder.fit_transform(data['BestTreeSpecies'])
data.head()

,Reported.Year,Property.ID..,Commercial.Property..Property.Street.Address,Building.Name,Total.SqFt,Category,Year.Built,kWh.sqft,Percentile.Rank,Received.Date,...,GeoLocation,Submitted.Name.of.Organization,GeoLocation_clean,lat,long,NearbyTreeSpecies,cluster,BestTreeSpecies,Category_encoded,BestTreeSpecies_encoded
0,2017.0,187746,"5126 E 5 ST, AUSTIN TX,","5126 E 5 ST, AUSTIN TX,",7419.0,NaN,1940,0.000,NaN,NaN,...,"5126 E 5 ST\nAUSTIN, TX\n(30.251707, -97.697313)","Pantur, Inc.","30.251707, -97.697313",30.251707,-97.697313,"Willow, Oak, Hackberry, Mesquite, Oak, Oak, Ha...",2,Pecan,27,20
1,2017.0,230740,"2500 RIDGEPOINT DR, AUSTIN TX, 78728","2500 RIDGEPOINT DR, AUSTIN TX, 78728",49024.0,INDUSTRIAL (W/sqft > 3),1983,15.580,39.58,NaN,...,"2500 RIDGEPOINT DR\nAUSTIN, TX 78728\n(30.3280...",Equitable Commercial Realty,"30.328069, -97.672401",30.328069,-97.672401,Oak,2,Oak,10,18
2,2017.0,729528,"1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741","1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741",101953.0,STORE\GROCERY (W/sqft > 5),2009,32.063,76.26,NaN,...,"1700 S PLEASANT VALLEY RD\nAUSTIN, TX 78741\n(...","Mimco, Inc.","30.236494, -97.721013",30.236494,-97.721013,"Crape Myrtle, Oak, Oak, Crape Myrtle, Crape My...",1,Oak,24,18
3,2017.0,526381,2011 W RUNDBERG LN Austin TX 78758,2011 W RUNDBERG LN Austin TX 78758,146142.0,OFFICE LG > 35000,1999,8.049,4.79,NaN,...,"2011 W RUNDBERG LN\nAustin, TX 78758\n(30.3737...",AISD,"30.373732, -97.719236",30.373732,-97.719236,"Oak, Oak, Pecan, Pecan, Sycamore, Pear, Pear, ...",2,Oak,16,18
4,2017.0,200001,"202 W 17 ST, AUSTIN TX, 78701","202 W 17 ST, AUSTIN TX, 78701",15350.0,OFFICE MED 10-35,1998,100.129,NaN,NaN,...,"202 W 17 ST\nAUSTIN, TX 78701\n(30.279196, -97...",Charter Communications,"30.279196, -97.740475",30.279196,-97.740475,"Pecan, Oak, Crape Myrtle, Crape Myrtle, Pecan,...",2,Oak,18,18


Replace year values with NaN

In [80]:
import numpy as np

np.random.seed(42)

# Calculate the empirical probability distribution for 1995-2001
year_counts = data[(data['Year.Built'] >= '1995') &
                   (data['Year.Built'] <= '2001') &
                   (data['Year.Built'] != '1995-2001')]['Year.Built'].value_counts()
year_probs = year_counts / year_counts.sum()

# Impute with weighted probabilities
data.loc[data['Year.Built'] == '1995-2001', 'Year.Built'] = data.loc[data['Year.Built'] == '1995-2001', 'Year.Built'].apply(lambda x: np.random.choice(year_probs.index, p=year_probs.values))

data['Year.Built'] = data['Year.Built'].replace('.', np.nan) # Impute with mode
data['Year.Built'] = data['Year.Built'].apply(lambda x: data['Year.Built'].mode()[0] if pd.isnull(x) or len(x) != 4 else x)

In [81]:
data['Year.Built'].unique()

array(['1940', '1983', '2009', '1999', '1998', '2001', '2005', '1978',
       '1985', '1973', '2004', '1996', '2008', '2015', '1997', '1976',
       '1960', '1971', '1953', '1955', '2013', '1910', '1987', '2000',
       '1975', '1970', '1986', '2007', '1972', '1981', '1942', '1979',
       '1959', '1969', '2006', '2010', '1989', '1927', '1939', '1991',
       '1984', '1963', '1982', '1947', '2011', '1925', '1935', '1952',
       '1954', '2014', '2016', '1980', '1988', '1974', '1968', '1966',
       '1965', '1946', '1994', '1967', '1977', '1993', '1962', '2002',
       '1964', '1875', '1958', '1992', '1914', '1880', '1920', '2003',
       '1951', '1956', '1936', '1995', '1887', '1930', '1957', '1948',
       '1945', '2012', '1871', '1922', '1931', '1878', '1949', '1950',
       '1990', '1900', '1934', '1908', '1926', '1852', '1961', '1892',
       '1937', '1923', '1929', '1944', '1915', '1924'], dtype=object)

Impute NA (do this with nearest neighbors or something later)

In [82]:
# Fill na
features = ['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'cluster']
na_count_before = data[features].isna().sum().sum()
n = data.shape[0]
print(f'NA Value count in X before imputation: {na_count_before} | Proportion of total: {na_count_before / n:.2f}')

## Continuous features get filled with median
data['Total.SqFt'] = data['Total.SqFt'].fillna(data['Total.SqFt'].median())
data['kWh.sqft'] = data['kWh.sqft'].fillna(data['kWh.sqft'].median())

# Categorical features get filled with mode
# data['Year.Built'] = data['Year.Built'].fillna(data['Year.Built'].mode()[0]) # Handled above
data['Category_encoded'] = data['Category_encoded'].fillna(data['Category_encoded'].mode()[0])

print(f'NA value count in X after imputation: {data[features].isna().sum().sum()}')

NA Value count in X before imputation: 833 | Proportion of total: 0.30
NA value count in X after imputation: 0


Remove species that don't appear often. Say 5 times.

In [83]:
# Remove classes with fewer than 25 instances
class_counts = data['BestTreeSpecies_encoded'].value_counts()
rare_classes = class_counts[class_counts < 25].index
data = data[~data['BestTreeSpecies_encoded'].isin(rare_classes)]
print(f'Shape of data after filtering: {data.shape}')
data.head()

Shape of data after filtering: (2677, 24)


,Reported.Year,Property.ID..,Commercial.Property..Property.Street.Address,Building.Name,Total.SqFt,Category,Year.Built,kWh.sqft,Percentile.Rank,Received.Date,...,GeoLocation,Submitted.Name.of.Organization,GeoLocation_clean,lat,long,NearbyTreeSpecies,cluster,BestTreeSpecies,Category_encoded,BestTreeSpecies_encoded
0,2017.0,187746,"5126 E 5 ST, AUSTIN TX,","5126 E 5 ST, AUSTIN TX,",7419.0,NaN,1940,0.000,NaN,NaN,...,"5126 E 5 ST\nAUSTIN, TX\n(30.251707, -97.697313)","Pantur, Inc.","30.251707, -97.697313",30.251707,-97.697313,"Willow, Oak, Hackberry, Mesquite, Oak, Oak, Ha...",2,Pecan,27,20
1,2017.0,230740,"2500 RIDGEPOINT DR, AUSTIN TX, 78728","2500 RIDGEPOINT DR, AUSTIN TX, 78728",49024.0,INDUSTRIAL (W/sqft > 3),1983,15.580,39.58,NaN,...,"2500 RIDGEPOINT DR\nAUSTIN, TX 78728\n(30.3280...",Equitable Commercial Realty,"30.328069, -97.672401",30.328069,-97.672401,Oak,2,Oak,10,18
2,2017.0,729528,"1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741","1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741",101953.0,STORE\GROCERY (W/sqft > 5),2009,32.063,76.26,NaN,...,"1700 S PLEASANT VALLEY RD\nAUSTIN, TX 78741\n(...","Mimco, Inc.","30.236494, -97.721013",30.236494,-97.721013,"Crape Myrtle, Oak, Oak, Crape Myrtle, Crape My...",1,Oak,24,18
3,2017.0,526381,2011 W RUNDBERG LN Austin TX 78758,2011 W RUNDBERG LN Austin TX 78758,146142.0,OFFICE LG > 35000,1999,8.049,4.79,NaN,...,"2011 W RUNDBERG LN\nAustin, TX 78758\n(30.3737...",AISD,"30.373732, -97.719236",30.373732,-97.719236,"Oak, Oak, Pecan, Pecan, Sycamore, Pear, Pear, ...",2,Oak,16,18
4,2017.0,200001,"202 W 17 ST, AUSTIN TX, 78701","202 W 17 ST, AUSTIN TX, 78701",15350.0,OFFICE MED 10-35,1998,100.129,NaN,NaN,...,"202 W 17 ST\nAUSTIN, TX 78701\n(30.279196, -97...",Charter Communications,"30.279196, -97.740475",30.279196,-97.740475,"Pecan, Oak, Crape Myrtle, Crape Myrtle, Pecan,...",2,Oak,18,18


In [84]:
# Write out prepped data for model fitting
data.to_csv('modeling_data.csv')

In [85]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd

# Load data
data = pd.read_csv('modeling_data.csv')
X = data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'cluster']].to_numpy()
y = data['BestTreeSpecies_encoded'].to_numpy()

# Data splitting
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5, stratify=y_, random_state=42)

# Set up StratifiedKFold for cross-validation on the train set
K = 3
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

# Hyperparameter grid for tuning
param_grid = {
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10]
}

# Initialize classifiers with criteria
clf_gini = DecisionTreeClassifier(criterion='gini', random_state=42)
clf_ent = DecisionTreeClassifier(criterion='entropy', random_state=42)

# Grid search on validation set for each criterion
grid_search_gini = GridSearchCV(clf_gini, param_grid, cv=skf, scoring='accuracy')
grid_search_ent = GridSearchCV(clf_ent, param_grid, cv=skf, scoring='accuracy')

# Fit on training data
grid_search_gini.fit(X_train, y_train)
grid_search_ent.fit(X_train, y_train)

print(f'Best params (gini): {grid_search_gini.best_params_}')
print(f'Best params (entropy): {grid_search_ent.best_params_}')

# Best models after tuning
best_clf_gini = grid_search_gini.best_estimator_
best_clf_ent = grid_search_ent.best_estimator_

# Evaluate on validation set
accuracy_gini_val = accuracy_score(y_val, best_clf_gini.predict(X_val))
accuracy_ent_val = accuracy_score(y_val, best_clf_ent.predict(X_val))

print(f"Validation Accuracy for Decision Tree (gini): {accuracy_gini_val * 100:.2f}%")
print(f"Validation Accuracy for Decision Tree (entropy): {accuracy_ent_val * 100:.2f}%")

# Check if model predicts only majority class on test set
y_pred_test_gini = best_clf_gini.predict(X_test)
y_pred_test_ent = best_clf_ent.predict(X_test)
print("Decision Tree (gini) Test Classification Report:")
print("Decision Tree (entropy) Test Classification Report:")

# Generate probabilistic predictions on test set for model calibration
y_proba_test_gini = best_clf_gini.predict_proba(X_test)
y_proba_test_ent = best_clf_ent.predict_proba(X_test)

# Print average accuracy for reference
print(f"Test Accuracy for Decision Tree (gini): {accuracy_score(y_test, y_pred_test_gini) * 100:.2f}%")
print(f"Test Accuracy for Decision Tree (entropy): {accuracy_score(y_test, y_pred_test_ent) * 100:.2f}%")

Best params (gini): {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best params (entropy): {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Validation Accuracy for Decision Tree (gini): 85.82%
Validation Accuracy for Decision Tree (entropy): 88.56%
Decision Tree (gini) Test Classification Report:
Decision Tree (entropy) Test Classification Report:
Test Accuracy for Decision Tree (gini): 84.33%
Test Accuracy for Decision Tree (entropy): 87.56%


In [48]:
data['BestTreeSpecies'].value_counts()

BestTreeSpecies
Oak                 1887
Elm                  254
Yaupon Holly          93
Pecan                 81
Ashe Juniper          72
Ash                   66
Huisache              62
Sugarberry            42
Crape Myrtle          34
Common Hackberry      30
Willow                29
Sycamore              27
Flame-Leaf Sumac      24
Mulberry              18
Cedar                 17
Hackberry             13
Brush                 11
Pistache               9
Cypress                7
Chinaberry             6
Mesquite               5
Goldenrain Tree        5
Name: count, dtype: int64